In [1]:
import sys
sys.path.append('..')
import gym

import torch
import torch.nn as nn


from models.td3 import TD3

In [2]:
class Actor(nn.Module):
    def __init__(self, nstates: int, nactions: int):
        super(Actor, self).__init__()
                
        self.fc1 = nn.Linear(nstates, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, nactions)
        
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        
        return x

In [3]:
class Critic(nn.Module):
    def __init__(self, nstates: int, nactions: int):
        super(Critic, self).__init__()
                
        self.fc1 = nn.Linear(nstates + nactions, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, 1)
        
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
                
    def forward(self, xs):
        x, a = xs
        
        x = self.relu1(self.fc1(torch.cat([x, a], 1)))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [4]:
lr = 1e-3
tau = 5e-3
eps = 0
gamma = 0.99

n_sts = 17
n_acts = 6

n_epis = 500
n_epoch = 200
n_rollout = 10

In [5]:
act = Actor(n_sts, n_acts)
act_opt = torch.optim.Adam(act.parameters(), lr=lr)

cri1 = Critic(n_sts, n_acts)
cri_opt1 = torch.optim.Adam(cri1.parameters(), lr=lr)

cri2 = Critic(n_sts, n_acts)
cri_opt2 = torch.optim.Adam(cri2.parameters(), lr=lr)

In [6]:
env = gym.make('HalfCheetah-v4', render_mode='human')

In [7]:
model = TD3(env, n_acts, act, act_opt, cri1, cri_opt1, cri2, cri_opt2, eps=eps, tau=tau, act_noise=0.2, act_range = (-1, 1))

In [8]:
model.train(n_epis, n_epoch, n_rollout)

c:\Users\esder\AppData\Local\Programs\Python\Python38\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


step: 210, score: -4.944132703620477, n_buffer: 210
step: 410, score: -6.318639933376945, n_buffer: 410
step: 610, score: -4.713585622154173, n_buffer: 610
step: 810, score: -4.853018845298598, n_buffer: 810
step: 1010, score: -4.995964120696923, n_buffer: 1010
step: 1210, score: -5.119522098733032, n_buffer: 1210
step: 1410, score: -3.389104401439616, n_buffer: 1410
step: 1610, score: -5.5679569416562105, n_buffer: 1610
step: 1810, score: -6.880888594630894, n_buffer: 1810
step: 2210, score: -0.946696132012222, n_buffer: 2210
step: 2410, score: -4.6332134517927495, n_buffer: 2410
step: 2610, score: -5.055369580994115, n_buffer: 2610
step: 2810, score: -5.120038370322496, n_buffer: 2810
step: 3010, score: -4.762137480969136, n_buffer: 3010
step: 3210, score: -3.798755828498251, n_buffer: 3210
step: 3410, score: -4.386222628675155, n_buffer: 3410
step: 3610, score: -3.4389229716813694, n_buffer: 3610
step: 3810, score: -1.6519790350983918, n_buffer: 3810
step: 4210, score: -2.2342092555

KeyboardInterrupt: 

In [9]:
env.close()